In [1]:
import pandas as pd
import re

In [7]:
file = r'Q:\Payroll\Payroll 2\client\current pay sheet.xlsx'
# make this say current time sheet 
x1 = pd.ExcelFile(file)
sheet_names = x1.sheet_names
ssn_mapping_df = pd.read_excel('EmployeeListExport.xlsx')
ssn_mapping_df = ssn_mapping_df[['Employee ID', 'SSN','Pay Group']]
ssn_mapping_df.rename(columns={'Pay Group': 'PayGroup'}, inplace=True)
ssn_mapping_df['ssn_last_four'] = ssn_mapping_df.SSN.str[-4:]
ssn_mapping_df.to_csv('ssn.csv')
df_arr = []

for sheet_name in sheet_names:
    if re.match(r'^CO\d{3}$',sheet_name):
        df = pd.read_excel(file,sheet_name,header=3)
        df = df[0:-1]
        df = df[['Unnamed: 1', 'Regular Hours Worked',
       'Vacation Hours', 'Overtime Hours Worked', 'Pay Period Hourly rate','Overtime Hourly Rate','Retail Bonus Pay',
       'Bonus', 'Service Bonus Pay', 'Tips ']]
        df['Retail Bonus Pay'].fillna(0, inplace=True)
        df['Service Bonus Pay'].fillna(0, inplace=True)
        df['Bonus'].fillna(0, inplace=True)
        df['E_COMM_DOLLARS'] = df['Retail Bonus Pay'] + df['Service Bonus Pay']
        df.columns = ['social_last_four','E_REG_Hours','E_PTO_HOURS','E_OT_HOURS','Rate','OT_Rate' ,'#Retail Bonus','E_BONUS_DOLLARS','#Service bonus pay','E_TIPSCC_DOLLARS','E_COMM_Dollars']
        df['E_REG_ORRate'] = df['Rate']
        df['E_PTO_ORRate'] = df['Rate']
        df['E_SICK-_ORRate'] = df['Rate']
        df['E_OT_ORRate'] = df['OT_Rate']
        df.fillna('0000',inplace=True)
        df['social_last_four'] = df['social_last_four'].astype(int)
        df['social_last_four'] = df['social_last_four'].astype(str)
        df['social_last_four'] = df['social_last_four'].apply(lambda x:'0'+x if len(x)==3 else x)
        df = df.merge(ssn_mapping_df, left_on='social_last_four', right_on='ssn_last_four', how='left')
        df['Employee ID'].fillna('000',inplace=True)
        df['key'] = df['Employee ID'].astype(int).astype(str)
        df_arr.append(df)
        df.to_csv(f'{sheet_name}_time_sheet_import.csv')
        
    
full_df = pd.concat(df_arr)    
full_df.to_csv(f'full_time_sheet_import.csv')
# TODO: Plug into specific folder all files
# TODO: change file locations to shared folder or to Q folder
# aTODO: Adjust paygroup to be the paygroup assigned in the sheet - sheet named CO103 - need the paygroup that co103 belongs too 
# TODO: make sure 0 is in from of numbers in last four of social 
